In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tenseal as ts
import torch
import torch.nn as nn

In [ ]:

def random_data(m=1024, n=1):
    x_train = torch.rand(m, n)
    x_test = torch.rand(m // 2, n)
    y_train = (x_train[:, 0] > 0.5).float().unsqueeze(0).t()
    y_test = (x_test[:, 0] > 0.5).float().unsqueeze(0).t()
    return x_train, y_train, x_test, y_test


In [ ]:
x_train, y_train, x_test, y_test = random_data(m=10000)


class SampleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1, 16)
        self.layer2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.sigmoid(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x


sample_model = SampleModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(sample_model.parameters())
batch_size = 10
data = torch.split(x_train, batch_size)
y = torch.split(y_train, batch_size)

for epoch in range(5):
    for i in range(len(data)):
        optimizer.zero_grad()
        output = sample_model(data[i])
        loss = loss_fn(output, y[i])
        loss.backward()
        optimizer.step()
        accuracy = (output.round() == y[i]).float().mean()
        print('Epoch {} Loss {} Accuracy {}'.format(epoch, loss.item(), accuracy))

with torch.no_grad():
    output_test = sample_model(x_test)
    accuracy_test = (output_test.round() == y_test).float().mean()
    print('Test Accuracy:', accuracy_test.item())

In [ ]:

def test_layer_by_layer_model(sample_model, data):
    layer1_weight = sample_model.layer1.weight.data.T
    layer1_bias = sample_model.layer1.bias.data
    layer2_weight = sample_model.layer2.weight.data.T
    layer2_bias = sample_model.layer2.bias.data
    for data_point in data:
        layer_1 = nn.Sigmoid()(data_point.mm(layer1_weight) + layer1_bias)
        layer_2 = nn.Sigmoid()(layer_1.mm(layer2_weight) + layer2_bias)
        assert layer_2 == sample_model(data_point)

In [ ]:
data = torch.split(x_test, 1)

test_layer_by_layer_model(sample_model, data)


# Encrypting data


In [ ]:
# Create the TenSEAL security context
def create_ctx():
    """Helper for creating the CKKS context.
    CKKS params:
        - Polynomial degree: 8192.
        - Coefficient modulus size: [40, 21, 21, 21, 21, 21, 21, 40].
        - Scale: 2 ** 21.
        - The setup requires the Galois keys for evaluating the convolutions.
    """
    bits_scale = 26

    poly_mod_degree = 16384

    coeff_mod_bit_sizes = [31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale,
                           bits_scale, 31]
    ctx = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    ctx.global_scale = pow(2, bits_scale)
    ctx.generate_galois_keys()

    # We prepare the context for the server, by making it public(we drop the secret key)
    server_context = ctx.copy()
    server_context.make_context_public()

    return ctx, server_context



In [ ]:
# Helper for encoding the image
def prepare_input(ctx, plain_input):
    enc_input = ts.ckks_vector(ctx, plain_input)
    return enc_input


def prepare_input_encrypted(context: bytes, ckks_vector: bytes) -> ts.CKKSVector:
    try:
        ctx = ts.context_from(context)
        enc_x = ts.ckks_vector_from(ctx, ckks_vector)
    except:
        raise ValueError("cannot deserialize context or ckks_vector")
    try:
        _ = ctx.galois_keys()
    except:
        raise ValueError("the context doesn't hold galois keys")
    return enc_x



In [ ]:
context, server_context = create_ctx()


In [ ]:
plt.hist(
    np.abs(np.array(prepare_input(context, torch.flatten(x_test)).decrypt()) - np.array(torch.flatten(x_test))))
plt.xlabel("Error")
plt.ylabel("Count")
plt.title("Single number encryption error")
plt.savefig("enc_err")




# encrypted model

In [ ]:
layer1_weight = sample_model.layer1.weight.data.T
layer1_bias = sample_model.layer1.bias.data
layer2_weight = sample_model.layer2.weight.data.T
layer2_bias = sample_model.layer2.bias.data
        # layer_1 = nn.Sigmoid()(data_point.mm(layer1_weight) + layer1_bias)
        # layer_2 = nn.Sigmoid()(layer_1.mm(layer2_weight) + layer2_bias)
x_train, y_train, x_test, y_test = random_data(m=1000)

# Decryption of result
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
errors = []
mean_err = []
for data, target in zip(x_train, y_train):
    # Encoding and encryption
    x_enc = prepare_input(context, data)
    enc_x = x_enc.mm(layer1_weight) + layer1_bias
    enc_x = nn.Sigmoid()(torch.tensor(enc_x.decrypt()))
    enc_x = prepare_input(context, enc_x)
    # enc_x = -0.004*enc_x*enc_x*enc_x+ 0.197*enc_x + 0.5
    enc_output = enc_x.mm(layer2_weight) + layer2_bias
    # enc_output = -0.004*enc_x*enc_x*enc_x+ 0.197*enc_x + 0.5
    # Decryption of result
    output = enc_output.decrypt()
    output = nn.Sigmoid()(torch.tensor(output))

    output = torch.tensor(output).view(1, -1)
    not_enc_model_output = sample_model(torch.tensor(data).type(torch.float))
    mean_err.append(torch.abs(not_enc_model_output - output))
    # compute loss
    if (not_enc_model_output > 0.5) == (output > 0.5):
        errors.append(1)
    else:
        errors.append(0)
print(f"Accuracy for binary prediction : {sum(errors) / len(errors)}")
mean_err = sum(mean_err) / len(x_train)
print(f"Mean Error per value for prediction {mean_err}")